In [72]:
# Necessary imports
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [73]:
# Load the data
df = pd.read_csv('data/only2_big_with_sf.csv')
# df = df.transpose()
df.set_index('cell_line', inplace=True)
# df = df.transpose()
display(df.head())

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,57185,81887,22875,6405,1080,54467,889,5893,572,51056,4267,9957,26,51384,23072,8379,3927,29916,55365,4074,90293,56603,3382,79007,57679,843,8837,7035,55471,10181,9108,6542,381,23098,26073,23129,204,952,2288,23028,10180,84254,5965,55610,126393,115703,4706,5166,51364,340273,170302,10165,996,799,3052,1856,64063,5976,8935,292,55013,3207,5439,56919,4222,221981,3980,79657,6296,81691,57019,56928,80256,51226,5577,10943,1387,9256,4353,5444,6936,84058,54677,5244,55904,57414,30812,25998,7748,23077,26224,3683,5164,3675,7543,3920,3674,51666,51573,55049,9244,114881,26022,51087,9020,6833,54972,163,6376,64847,8913,51330,1750,4293,5898,55971,80853,2115,55750,221,55001,57157,10344,9855,10083,79017,10926,6915,3475,1352,9569,54623,27072,9912,60528,85477,414,8228,79602,51265,10942,57787,8842,115098,5048,4843,1770,9703,8522,79090,4145,974,6329,54879,8277,5080,113000,64285,55692,9254,8938,115939,9091,57585,7005,55735,2328,29116,27035,1870,5689,6853,3720,6792,57172,65220,9940,9265,53616,6856,1534,9043,1951,10157,55200,51188,84954,4257,1407,8993,4784,6482,64386,9235,23542,9862,23221,54497,7095,7812,9690,5980,10178,10747,389434,199870,80342,29954,51534,51085,9031,10048,51513,83932,51603,8798,7756,7384,83930,928,9918,25900,2696,51533,56920,11188,23166,80199,6540,3423,5646,7752,920,10233,695,3077,22955,2534,3097,6924,...,552889,5591,9708,56106,137075,339324,4067,54984,56099,56112,91056,132430,4975,8790,92312,5003,84769,729991,10072,3846,55845,57132,493861,23741,79400,55096,91893,9865,1519,84334,161582,9668,100507203,7556,90649,85027,3145,7768,373863,11232,100129361,59348,56242,388327,84290,100129128,57189,26065,283948,283459,8972,90589,255783,10330,93432,1523,23601,91612,51750,147660,246777,6038,4157,90799,10381,388021,100130933,3690,100533106,79178,26589,146429,646627,29890,3217,399693,4360,94086,100506581,343702,9236,100507421,83481,100507290,5175,100507050,8139,145788,83482,100129654,56107,79585,4321,56111,10418,6182,92002,2969,10780,100820829,10929,9641,388677,140735,56957,2702,5949,9939,100287932,27246,118738,9554,10628,23380,57597,83593,92335,136259,8031,54460,80022,5348,342892,9294,112812,7767,100506374,440335,55957,4701,56936,55879,11254,147670,125972,388630,115290,79917,8517,6689,84548,401612,152217,8148,246176,146853,91608,140809,79148,6352,128077,284021,389332,284615,404672,11165,9757,100133941,154313,152687,8447,57717,55802,150590,2904,728116,54883,100170841,9326,8342,3975,23191,389856,79159,8355,256302,30837,151,8344,81554,8348,8353,140883,4302,57223,11276,79873,7549,94059,8330,157285,8354,8857,6928,26574,8345,8350,57636,114791,5645,11072,29128,84876,8294,146850,6871,8396,8331,56101,338382,1602,8332,8367,3012,8360,284098,7703,80725,4082,93474,338328,4747,8968,5691,6716,284106,201299,11056,7554,6753,80179,79893,7772,8335,284099,79154,31,10002,79922,8359,8336,84311,126661,79447,283521,29122,9997,388963,6905,SF
cell_line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NCIH1568-2,5.626439,6.855242,2.185867,4.079805,0.084064,0.176323,6.824513,4.956521,3.766595,2.298658,3.160275,4.706531,2.260026,3.750607,0.056584,5.184677,5.652486,3.694880,6.048323,6.031660,8.531147,0.432959,0.084064,0.028569,0.000000,4.936873,6.662490,3.403268,0.000000,6.378858,0.475085,4.867402,5.780048,5.399855,3.674687,1.411426,4.709842,5.156235,4.029453,6.630522,0.687061,4.970394,6.792205,2.235727,6.229011,3.994580,7.446256,0.028569,6.517591,5.921246,5.692092,1.056584,4.755422,4.336997,0.321928,3.558268,7.087781,1.752749,0.895303,0.014355,0.356144,5.706807,4.863443,0.275007,6.023

In [74]:
scaler = MinMaxScaler()
df_scaled_x = scaler.fit_transform(df.iloc[:, :-1])
y_non_scaled = df.iloc[:, -1]
df_scaled_x = pd.DataFrame(df_scaled_x, columns=df.columns[:-1])
display(y_non_scaled.head())

cell_line
NCIH1568-2    0.607940
SCC9-2        0.573035
HCC1428-2     0.450188
U2OS-2        0.526517
LU99-2        0.625161
Name: SF, dtype: float64

In [75]:
X, y = df_scaled_x, y_non_scaled
X.head()

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,57185,81887,22875,6405,1080,54467,889,5893,572,51056,4267,9957,26,51384,23072,8379,3927,29916,55365,4074,90293,56603,3382,79007,57679,843,8837,7035,55471,10181,9108,6542,381,23098,26073,23129,204,952,2288,23028,10180,84254,5965,55610,126393,115703,4706,5166,51364,340273,170302,10165,996,799,3052,1856,64063,5976,8935,292,55013,3207,5439,56919,4222,221981,3980,79657,6296,81691,57019,56928,80256,51226,5577,10943,1387,9256,4353,5444,6936,84058,54677,5244,55904,57414,30812,25998,7748,23077,26224,3683,5164,3675,7543,3920,3674,51666,51573,55049,9244,114881,26022,51087,9020,6833,54972,163,6376,64847,8913,51330,1750,4293,5898,55971,80853,2115,55750,221,55001,57157,10344,9855,10083,79017,10926,6915,3475,1352,9569,54623,27072,9912,60528,85477,414,8228,79602,51265,10942,57787,8842,115098,5048,4843,1770,9703,8522,79090,4145,974,6329,54879,8277,5080,113000,64285,55692,9254,8938,115939,9091,57585,7005,55735,2328,29116,27035,1870,5689,6853,3720,6792,57172,65220,9940,9265,53616,6856,1534,9043,1951,10157,55200,51188,84954,4257,1407,8993,4784,6482,64386,9235,23542,9862,23221,54497,7095,7812,9690,5980,10178,10747,389434,199870,80342,29954,51534,51085,9031,10048,51513,83932,51603,8798,7756,7384,83930,928,9918,25900,2696,51533,56920,11188,23166,80199,6540,3423,5646,7752,920,10233,695,3077,22955,2534,3097,6924,...,342538,552889,5591,9708,56106,137075,339324,4067,54984,56099,56112,91056,132430,4975,8790,92312,5003,84769,729991,10072,3846,55845,57132,493861,23741,79400,55096,91893,9865,1519,84334,161582,9668,100507203,7556,90649,85027,3145,7768,373863,11232,100129361,59348,56242,388327,84290,100129128,57189,26065,283948,283459,8972,90589,255783,10330,93432,1523,23601,91612,51750,147660,246777,6038,4157,90799,10381,388021,100130933,3690,100533106,79178,26589,146429,646627,29890,3217,399693,4360,94086,100506581,343702,9236,100507421,83481,100507290,5175,100507050,8139,145788,83482,100129654,56107,79585,4321,56111,10418,6182,92002,2969,10780,100820829,10929,9641,388677,140735,56957,2702,5949,9939,100287932,27246,118738,9554,10628,23380,57597,83593,92335,136259,8031,54460,80022,5348,342892,9294,112812,7767,100506374,440335,55957,4701,56936,55879,11254,147670,125972,388630,115290,79917,8517,6689,84548,401612,152217,8148,246176,146853,91608,140809,79148,6352,128077,284021,389332,284615,404672,11165,9757,100133941,154313,152687,8447,57717,55802,150590,2904,728116,54883,100170841,9326,8342,3975,23191,389856,79159,8355,256302,30837,151,8344,81554,8348,8353,140883,4302,57223,11276,79873,7549,94059,8330,157285,8354,8857,6928,26574,8345,8350,57636,114791,5645,11072,29128,84876,8294,146850,6871,8396,8331,56101,338382,1602,8332,8367,3012,8360,284098,7703,80725,4082,93474,338328,4747,8968,5691,6716,284106,201299,11056,7554,6753,80179,79893,7772,8335,284099,79154,31,10002,79922,8359,8336,84311,126661,79447,283521,29122,9997,388963,6905
0,0.749472,0.467404,0.308779,0.622956,0.025473,0.020444,0.876804,0.420781,0.339090,0.504840,0.446351,0.418527,0.416010,0.507454,0.007639,0.680368,0.742049,0.540879,0.566657,0.661738,0.715423,0.073417,0.010689,0.007176,0.000000,0.393733,0.582965,0.243028,0.000000,0.466128,0.065319,0.681693,0.696654,0.790464,0.403322,0.289694,0.515842,0.506474,0.655780,0.759110,0.120594,0.651346,0.371421,0.342220,0.423291,0.455361,0.396842,0.006815,0.420146,0.487187,0.596147,0.163966,0.361342,0.712750,0.034070,0.506675,0.570007,0.201544,0.201211,0.001875,0.061074,0.706900,0.431655,0.059488,0.646481,0.500817,0.617694,0.803695,0.688690,0.770096,0.710214,0.270448,0.473731,0.614508,0.049564,0.018484,0.564317,0.460982,0.041927,0.588454,0.739516,0.672847,0.537559,0.006956,0.580773,0.440078,0.645041,0.030689,0.004140,0.002836,0.718003,0.568273,0.475128,0.002075,0.405731,0.788443,0.091178,0.730366,0.378634,0.363245,0.514864,0.069778,0.629604,0.585487,0.702708,0.654871,0.339582,0.004220,0.560293,0.825912,0.316681,0.048739,0.443364,0.455656,0.338612,0.076058,0.777467,0.391322,0.001947,0.406381,0.003131,0.670222

In [76]:
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.1, random_state = 123)

In [77]:
params = { 'max_depth': [3,5,7],
           'learning_rate': [0.01, 0.05],
           'n_estimators': [50, 100],
           'colsample_bytree': [0.3, 0.5]}

In [78]:
xgbr = xgb.XGBRegressor(seed = 20, tree_method='gpu_hist')

In [79]:
clf = GridSearchCV(estimator=xgbr,
                   param_grid=params,
                   scoring='neg_mean_squared_error',
                   verbose=10, n_jobs=1)

In [80]:
clf.fit(train_X, train_y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5; 1/24] START colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50
[CV 1/5; 1/24] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50;, score=-0.045 total time=   1.7s
[CV 2/5; 1/24] START colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50
[CV 2/5; 1/24] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50;, score=-0.049 total time=   1.5s
[CV 3/5; 1/24] START colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50
[CV 3/5; 1/24] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50;, score=-0.046 total time=   1.5s
[CV 4/5; 1/24] START colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50
[CV 4/5; 1/24] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=50;, score=-0.037 total time=   1.5s
[CV 5/5; 1/24] START colsample_bytree=0.3, learning_rate=0.01, max_depth=3

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_o...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             n_jobs=1,
             param_grid={'colsample_bytree': [0.3, 0.5],
                         'learning_rate': [0.01, 0.05], 'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100]},
             scoring='neg_mean_squared_error', verbose=10)

In [81]:
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

#Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}
# Lowest RMSE:  0.19178145326877513

Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}
Lowest RMSE:  0.19178145326877513
